In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from time import time
from time import sleep
from random import randint

#Setting driver
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
options.add_argument('--disable-gpu')

#Creating driver
driver = webdriver.Chrome(options=options)

job_search = 'data'
location_search = 'france'
url = f"https://indeed.com/jobs?q={job_search}&l={location_search}&limit=50&start="

driver.get(url)

sleep(randint(5,8))
print(f'Collecting data for "{job_search}"...') # First, get the number of jobs available


#We find how many jobs are offered
jobs_num = driver.find_element(By.CLASS_NAME,"jobsearch-JobCountAndSortPane-jobCount").get_attribute("innerText")
jobs_num = jobs_num.strip("\xa0 jobs") #xa0 jobs offers
jobs_num = jobs_num.replace(",", "")

job_number = int(jobs_num)
print(f"- Number of open positions : {job_number}")
exact_page_nb = job_number / 50
print(f"- Exact number of pages to be crawled : {exact_page_nb}")
min_page_nb = job_number // 50
print(f"- Minimum number of pages to be crawled : {min_page_nb}")

if exact_page_nb > min_page_nb:
    page_nb = (min_page_nb ) * 50
    pages = [str(i) for i in range(0, page_nb, 50)]
elif exact_page_nb == min_page_nb:
    page_nb = (min_page_nb - 1) * 50
    pages = [str(i) for i in range(0, page_nb, 50)]


job_link_list = []

for page in pages[:200]:
    driver.get(url + page)

    sleep(randint(5, 12))

     # The job browing is started
    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon")

    for job in jobs:
    # Finding the job title and its url
        job_title = job.find_element(By.CLASS_NAME,"jobTitle")
        job_link = job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
        #If not job_link in job_link_list :
        job_link_list.append(job_link)
print(f"- Number of offers scrapped : {len(job_link_list)}")

In [ ]:
import numpy as np
print(len(job_link_list))
print(len(np.unique(job_link_list)))


In [ ]:
linked = pd.DataFrame({"job_kink": job_link_list})
linked.to_csv('linked.csv', sep=';', encoding ='utf-8', index=False)

In [ ]:
title_job = []
company_job = []
location_job = []
type_job = []
desc_job = []

#Wait = WebDriverWait(driver, 60)


for link in job_link_list:
    driver.get(link)
    try:
        job_title_page = driver.find_element(By.CSS_SELECTOR, 'div.jobsearch-JobInfoHeader-title-container').text
        title_job.append(job_title_page)
    except NoSuchElementException:
        title_job.append("no offer")

    try:
        company_page = driver.find_element(By.CSS_SELECTOR, 'div.css-1h46us2.eu4oa1w0').text
        company_job.append(company_page)
    except NoSuchElementException:
        company_job.append("no company description")

    try:
        location_page = driver.find_element(By.CSS_SELECTOR, 'div.css-6z8o9s.eu4oa1w0').text
        location_job.append(location_page)
    except NoSuchElementException:
        location_job.append("no location description")

    try:
        type_page = driver.find_element(By.CSS_SELECTOR, 'div.css-804pn3.eu4oa1w0').text
        type_job.append(type_page)
    except NoSuchElementException:
        type_job.append("no type description")

    try:
        job_desc_page = driver.find_element(By.CSS_SELECTOR, 'div.jobsearch-jobDescriptionText').text
        desc_job.append(job_desc_page)
    except NoSuchElementException:
        desc_job.append("no release description")
    sleep(1)

#Driver.quit()

In [ ]:
# Writing to df
data = pd.DataFrame({"job_title": title_job,"job_company": company_job,"job_location": location_job, "job_type": type_job,"job_description": desc_job})
data.head()

In [ ]:
data.to_csv('data.csv', sep=';', encoding ='utf-8', index=False)

In [ ]:
data.shape